In [ ]:
import graphviz

# Define the graph
dot = graphviz.Digraph(comment='SEM Model')

# Adding nodes for latent variables (factors)
dot.node('F1', 'Factor 1')
dot.node('F2', 'Factor 2')

# Adding nodes for observed variables
dot.node('Y1', 'Y1')
dot.node('Y2', 'Y2')
dot.node('Y3', 'Y3')
dot.node('Y4', 'Y4')
dot.node('Y5', 'Y5')
dot.node('Y6', 'Y6')

# Adding edges for factor loadings
dot.edges(['F1Y1', 'F1Y2', 'F1Y3', 'F2Y4', 'F2Y5', 'F2Y6'])

# Adding an edge for the relationship between latent factors
dot.edge('F1', 'F2', label='0.5')

# Render the graph to a file (e.g., PDF, PNG)
dot.render('sem_model', format='png', cleanup=True)

# To display the graph here, let's convert it to an image and then display it
dot
